<a href="https://colab.research.google.com/github/adeshmukh/gaiip/blob/main/notebooks/Intro_to_Guardrails_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://www.guardrailsai.com/docs/guardrails_ai/getting_started
More examples at: https://github.com/guardrails-ai/guardrails/tree/main/docs/examples

In [1]:
%%capture

!pip install orjson==3.10.6
!pip install guardrails-ai==0.5.1

In [2]:
from google.colab import userdata
import os

os.environ["GUARDRAILSAI_API_KEY"] = userdata.get('GUARDRAILSAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Configure the GuardRails hub

In [3]:
!guardrails configure --enable-metrics --enable-remote-inferencing --token "${GUARDRAILSAI_API_KEY}"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

            Login successful.

            Get started by installing our RegexMatch validator:
            https://hub.guardrailsai.com/validator/guardrails_ai/regex_match

            You can install it by running:
            guardrails hub install hub://guardrails/regex_match

            Find more validators at https://hub.guardrailsai.com
            


In [ ]:
!guardrails hub install hub://guardrails/regex_match       --quiet --install-local-models
!guardrails hub install hub://guardrails/valid_length      --quiet --install-local-models
!guardrails hub install hub://guardrails/toxic_language    --quiet --install-local-models
# !guardrails hub install hub://guardrails/unusual_prompt    --quiet --install-local-models
# !guardrails hub install hub://guardrails/reading_time      --quiet --install-local-models
# !guardrails hub install hub://guardrails/politeness_check  --quiet --install-local-models
# !guardrails hub install hub://guardrails/profanity_free    --quiet --install-local-models
# !guardrails hub install hub://guardrails/restricttotopic   --quiet --install-local-models
# !guardrails hub install hub://guardrails/competitor_check  --quiet --install-local-models
# !guardrails hub install hub://guardrails/detect_pii        --quiet --install-local-models
# !guardrails hub install hub://guardrails/financial_tone    --quiet --install-local-models

# Basic Guards

Guards can be used as a general-purpose validation framework. This use by itself is not very interesting. But it's still useful to know for a couple of reasons:
1. It is a gentle introduction to the concept of guards and their usage.
2. It's somewhat useful to apply uniform paradigm for all types of validations in a Gen AI application.

In [ ]:
# Import Guard and Validator
from guardrails.hub import RegexMatch
from guardrails import Guard

# Initialize the Guard with the regex
guard = Guard().use(
    RegexMatch(regex="^[A-Z][a-z]*$")
)

# Guardrail passes
validation_outcome = guard.parse("Caesar")
print(validation_outcome.model_dump_json(indent=2))

# Guardrail fails
validation_outcome = guard.parse("Caesar is a great leader")
print(validation_outcome.model_dump_json(indent=2))

# Generating structured JSON output from LLM

Often, in agentic interactions with LLM, it is desirable to ensure that the output conforms to a JSON structure. This can be achieved in a couple of different ways using Guardrails AI.

First, let's set up a simple pydantic model which will be used to constrain the LLM response.

In [ ]:
from pydantic import BaseModel, Field

class Pet(BaseModel):
    pet_type: str = Field(description="Species of pet")
    name: str = Field(description="a unique pet name")

## Using Pydantic support in GuardrailsAI

Encapsulating the LLM interaction with Guard. Guardrails has native support for pydantic objects.

Note that this uses a prompt template with a [built-in constant](https://github.com/guardrails-ai/guardrails/blob/main/guardrails/constants.xml).

In [ ]:
from guardrails import Guard

# NOTE: This is a prompt "template" with a parameter named "gr.complete_xml_suffix_v2", which resolves to a built-in constant.
# Built-in constants are defined in guardrails/constants.xml
prompt = """
    What kind of pet should I get and what should I name it?
    I would like to have a pet that I can take with me to places I visit.

    ${gr.complete_xml_suffix_v2}
"""
guard = Guard.from_pydantic(output_class=Pet)

res = guard(
    messages=[{"role": "user", "content": prompt}],
    model="gpt-3.5-turbo"
)

print(res.validated_output)

## JSON output for tool invocations



In [ ]:
from pydantic import BaseModel, Field
from typing import List
from guardrails import Guard

class Fruit(BaseModel):
    name: str
    color: str

class Basket(BaseModel):
    fruits: List[Fruit]

guard = Guard.from_pydantic(Basket)


In [ ]:
result = guard(
    messages=[{"role":"user", "content":"Generate a basket of 5 fruits"}],
    model="gpt-4o",
    tools=guard.json_function_calling_tool([]),
    tool_choice="required",
)

print(f"{result.model_dump_json(indent=2)}")

# Generate Structured Data
Ref: https://www.guardrailsai.com/docs/how_to_guides/generate_structured_data


In [ ]:
from guardrails import Guard
from guardrails.hub import RegexMatch
from pydantic import BaseModel, Field
from typing import List

NAME_REGEX = "^[A-Z][a-z]+\s[A-Z][a-z]+$"

class Delivery(BaseModel):
    customer_name: str=Field(validators=[RegexMatch(regex=NAME_REGEX)], description="customer name")
    pickup_time: str=Field(description="date and time of pickup")
    pickup_location: str=Field(description="address of pickup")
    dropoff_time: str=Field(description="date and time of dropoff")
    dropoff_location: str=Field(description="address of dropoff")
    price: str = Field(description="price of delivery with currency symbol included")

class Schedule(BaseModel):
    deliveries: List[Delivery]

guard = Guard.from_pydantic(Schedule)
chat_history="""
nelson and murdock: i need a pickup 797 9th Avenue, manila envelope, June 3 10:00am with dropoff 10:30am Courthouse, 61 Center Street C/O frank james
operator: quote - $23.00
neslon and murdock: perfect, we accept the quote
operator: 797 9th ave, 10:00am pickup comfirmed
abc flowers: i need a pickup of a flowers from abc flowers at 21 3rd street at 11:00am on june 2 with a dropoff at 75th Ave at 5:30pm same day
operator: 21 3rd street flowers quote - $14.50
abc flowers: accepted
polk and wardell: i need a pickup of a bagels from Bakers Co at 331 5th street at 11:00am on june 3 with a dropoff at 75th Ave at 5:30pm same day
operator: 331 5th street bagels quote - $34.50
polk and wardell: accepted
"""

# NOTE: This is a prompt template with parameter "chat_history".
prompt = """
From the chat exchanges below extract a schedule of deliveries.
Chats:
${chat_history}
"""

messages = [{
  "role": "system",
  "content": "You are a helpful assistant."
}, {
  "role": "user",
  "content": prompt
}]

In [ ]:
# Let's inspect the "prompt" and note that "chat_history" is not yet resolved.
print(prompt)

In [ ]:
# Now let's take a peek at the "messages" with the "prompt"
import json
print(json.dumps(messages, indent=2))

## Approach A: Using plain prompt engineering

In this approach, the guard will use prompt engineering to get the output to conform to JSON structure.

In [ ]:
# NOTE: We are extending the prompt template.
# Another template "gr.complete_json_suffix_v3" is added here.
prompt+="""

${gr.complete_json_suffix_v3}
"""
validation_response = guard(
    model="gpt-4o",
    messages=[messages[0],{
      "role": "user",
      "content": prompt
    }],
    prompt_params={"chat_history": chat_history},
)

In [ ]:
print(validation_response.model_dump_json(indent=2))

## Approach B: JSON mode

Some LLMs, like OpenAI provide native support for [JSON mode](https://platform.openai.com/docs/guides/json-mode). JSON mode ensures that the output from the LLM conforms to a valid JSON structure.

In [ ]:
messages[-1]["content"]+="""

${gr.complete_json_suffix_v3}
"""
response = guard(
    model="gpt-4o",
    messages=messages,
    prompt_params={"chat_history": chat_history},
    response_format={ "type": "json_object" }
)

## Approach C: Using function calling

Some LLMs, like OpenAI provide native support for [function calling](https://platform.openai.com/docs/guides/function-calling). And function calling uses JSON mode, thus ensuring that the LLM invokes a function *and* returns a JSON response.

In this case, we will configure the guard to use function calling.

In [ ]:
tools = [] # an open ai compatible list of tools

validation_response = guard(
    model="gpt-4o",
    messages=messages,
    prompt_params={"chat_history": chat_history},
    tools=guard.json_function_calling_tool(tools),
    tool_choice="required",
)

In [ ]:
print(validation_response.model_dump_json(indent=2))

## Guarding against toxic language


In [ ]:

# Import the guardrails package
# and the ToxicLanguage validator
# from Guardrails Hub
import guardrails as gd
from guardrails.hub import ToxicLanguage
from rich import print

# Create a Guard object with this validator
# Here, we'll use the default validation method of "sentence"
# and the default threshold of 0.25

guard = gd.Guard.from_string(
    validators=[ToxicLanguage(on_fail="fix")],
    description="testmeout",
)

In [ ]:

# Parse the raw response
raw_response = """
What a lovely day it has been to roam around the city. I love it here!
I hate how pathetic and needy you can be sometimes.
"""
raw_llm_output, validated_output, *rest = guard.parse(
    llm_output=raw_response,
)

# Print the output
print(validated_output)

In [ ]:

# Parse the raw response
raw_response = """
I am a Generative AI model that is trained on a large corpus of text.
I am shocked by how disgusting and vile you are.
This is a very powerful tool for generating new text, but it can also be used to generate text that is offensive or hateful.
"""
raw_llm_output, validated_output, *rest = guard.parse(
    llm_output=raw_response,
)

# Print the output
print(validated_output)

In [ ]:

# Test with validation method 'full'
full_guard = gd.Guard.from_string(
    validators=[ToxicLanguage(validation_method="full", on_fail="fix")],
    description="testmeout",
)

# Parse the raw response
raw_response = "Stop being such a dumb piece of shit. Why can't you comprehend this?"
raw_llm_output, validated_output, *rest = full_guard.parse(
    llm_output=raw_response,
)

# Print the output
print(validated_output)